In [1]:
from ultralytics import YOLO
import cv2
import os

In [6]:
class ObjectDet:
    def __init__(self, detection_score=0.6):
        ### Init model
        self.__setup_model()
        self.detection_score = detection_score
        self.area_focus = {'start': (250,420), 'end':(450,520)}

        ### Config
        self.is_draw = True
        self.class_dict = {0:'person', 1:'phone'}
        self.class_color_dict = {0:(255, 0, 0), 
                                 1:(0, 0, 255),
                                 'area_normal':(0,128,128),
                                 'area_alert':(0,165,255)
                                 }

    def __setup_model(self):
        model_path = os.path.join('../weight/cctv-feed-ml_v2_yolov8l.pt')
        self.model = YOLO(model_path)

    def __bb_intersection_over_person(self, box_focus, box_person):
        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(box_focus[0], box_person[0])
        yA = max(box_focus[1], box_person[1])
        xB = min(box_focus[2], box_person[2])
        yB = min(box_focus[3], box_person[3])

        # compute the area of intersection rectangle
        intersec_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

        # rectangles
        box_focus_area = (box_focus[2] - box_focus[0] + 1) * (box_focus[3] - box_focus[1] + 1)
        box_person_area = (box_person[2] - box_person[0] + 1) * (box_person[3] - box_person[1] + 1)

        iop = intersec_area / box_person_area #float(boxAArea + boxBArea - interArea) ### iop = intersec over person

        return iop


    def inference(self, img):
        alert_focus_area = False

        result = self.model(img, verbose=False)

        cls = result[0].boxes.cls.cpu().numpy()    # cls, (N, 1)
        probs = result[0].boxes.conf.cpu().numpy()  # confidence score, (N, 1)
        boxes = result[0].boxes.xyxy.cpu().numpy()   # box with xyxy format, (N, 4)

        lab_arr = []
        
        if self.is_draw:
            # print(cls,probs)
            for clas, prob, box in zip(cls, probs, boxes):
                if prob >= self.detection_score:

                    (x, y, x2, y2) = box

                    # print(clas, int(x), int(y), int(x2), int(y2))
                    lab_arr.append([clas, int(x), int(y), int(x2), int(y2)])

                    # cal iop - intersec over person
                    iop = self.__bb_intersection_over_person(box_focus = list(self.area_focus['start']) + list(self.area_focus['end']), # [x1,y1,x2,y2]
                                                             box_person = [x,y,x2,y2])
                    if iop >= 0.1:
                        alert_focus_area = True


                    # Draw rec
                    cv2.rectangle(img, (int(x), int(y)), (int(x2), int(y2)), self.class_color_dict[clas], 2) ## class
                    cv2.rectangle(img, self.area_focus['start'], self.area_focus['end'], color=self.class_color_dict['area_normal'], thickness=2) #(x1,y1),(x2,y2)


                    # Get size txt
                    draw_txt = self.class_dict[clas] + ": " + str(round(prob*100,1)) +"%"
                    (w, h), _ = cv2.getTextSize(
                            draw_txt, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

                    # Prints the text.    
                    cv2.rectangle(img, (int(x), int(y) - 20), (int(x) + w, int(y)), self.class_color_dict[clas], -1)
                    cv2.putText(img, draw_txt, (int(x), int(y) - 5),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
                    
                    ### show counting frame
                    # cv2.putText(img, '1', (int(20), int(30)),
                    #                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

                    
                    # cv2.putText(img, str(clas) + '_' + str(round(prob,3)), (int(x), int(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 1, self.class_color_dict[clas], 2)  # Text in red color

        return img, alert_focus_area, lab_arr

In [7]:
OD = ObjectDet(detection_score=0.6)

In [10]:
import json
from tqdm import tqdm

### edit this ###
class_map = {
            0.0: 'person',
            1.0: 'phone',
            # Add more class mappings as needed
        }

folder_root = '../../cctv-feed-outside-room/test-image/UAT_inspect'
folder_list = ['KBF','KTF','LPF','PLF','PTF','SRF']
### edit this ###


for focus_folder in folder_list:
    src_drt = f'{folder_root}/{focus_folder}/'
    files_list = os.listdir(src_drt) 


    files_name = [filename for filename in files_list if filename.endswith('.jpg')]
    for img_name in tqdm(files_name):
        img = cv2.imread(src_drt + img_name, cv2.IMREAD_COLOR)
        image_height, image_width, channels = img.shape

        det,alert_focus_area,lab_arr = OD.inference(img)

        labelme_annotations = []
        file_name = img_name.split('.')[0]

        for yolo_label in lab_arr:
            clas, x_top_left, y_top_left, x_bottom_right, y_bottom_right = yolo_label

            labelme_annotation = {
                    "label": class_map[clas],  # Replace with actual class name
                    "points": [[x_top_left, y_top_left], [x_bottom_right, y_bottom_right]],
                    "description": "",
                    "shape_type": "rectangle"
                }

            labelme_annotations.append(labelme_annotation)


        json_file_full = {
            "version": "5.3.0a0",
            "flags": {},
            "shapes" : labelme_annotations,
            "imagePath": file_name + '.jpg',
            "imageData" : None
        }


        # print(json_file_full)

        with open(src_drt + file_name + ".json", "w") as json_file:
            json.dump(json_file_full, json_file)

# ======================================

100%|██████████| 95/95 [01:10<00:00,  1.35it/s]


In [10]:
# print(img)